In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

from torch import nn, optim
from torch_tool.trainer import TorchModule

## Define bearing model

In [ ]:
class BearingNet(TorchModule):
    def __init__(self, in_channels = 1, encode_size = 128, use_gpu = True, gpu_id = 0):
        super().__init__(use_gpu = use_gpu, gpu_id = gpu_id)

        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels = in_channels, out_channels = 20, kernel_size = 5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2),

            nn.Conv2d(in_channels = 20, out_channels = 16, kernel_size = 4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2),

            nn.Conv2d(in_channels = 16, out_channels = 16, kernel_size = 3),
            nn.ReLU(),

            nn.Flatten()
        )

        self.fully_connected = nn.Sequential(
            nn.BatchNorm1d(144),
            nn.Linear(144, 132),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.BatchNorm1d(132),
            nn.Linear(132, encode_size),
            nn.Dropout(0.5),
            nn.Sigmoid()
        )

        self.criterion = nn.TripletMarginLoss()

    def forward(self, img):
        out = self.cnn(img)
        out = self.fully_connected(out)

        return out

## Read model

In [ ]:
# ------- Read facenet -------
import torch
from dataset import get_pattern
model = BearingNet()
model_path = './checkpoint/model_16_1.41.pth'
params = torch.load(model_path)
model.load_state_dict(params)

# -------- Read random forest -------
from joblib import load
RF = load('./checkpoint/RF_0.87.pth')

# ------- Read pattern vector -------
PatternVector = load('./data/Pattern_Vector/PatternVector.pth')

## Get encoding vector

In [ ]:
from dataset import get_dict

test_dict = get_dict(path = './data/Image/test')
class_lst = ['Normal', 'Inner_break', 'Outer_break', 'Ball']
label_lst = []
vec_lst = torch.zeros(1, 128)
count = 0

with torch.no_grad():
    for cla in class_lst:
        length = test_dict[cla].shape[0]
        label_lst += [count for _ in range(length)]
        vec = model(test_dict[cla])
        vec_lst = torch.cat([vec_lst, vec], dim = 0)
        count += 1

vec_lst_torch = vec_lst[1:]
vec_lst_np = vec_lst_torch.numpy()
label_lst = np.array(label_lst)

print(vec_lst_np.shape, label_lst.shape)

## Facenet test

In [ ]:
pred = np.zeros(label_lst.shape[0])
thrs = 1.125
for i, vec in enumerate(vec_lst_torch):
    count = 0
    for key in PatternVector.keys():
        dist = torch.sqrt(torch.sum((vec - PatternVector[key]) ** 2))
        
        if dist < thrs:
            count += 1
            break
    
    if count == 0:
        pred[i] = 3
    else:
        pred[i] = RF.predict(vec_lst_np[i].reshape(1, 128))[0]

In [ ]:
              precision    recall  f1-score    support

           0       0.71      0.61      0.68        100
           1       0.52      0.59      0.56        103
           2       0.55      0.58      0.59        155
           3       0.49      0.43      0.47        118

    accuracy                           0.53        476
   macro avg       0.88      0.88      0.54        476
weighted avg       0.89      0.87      0.55        476